# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [27]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [3]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
spaceship.dropna(inplace=True)


In [5]:
spaceship.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [6]:
spaceship["Cabin"] = spaceship["Cabin"].apply(lambda x: x[0])
spaceship

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [7]:
spaceship.drop(["PassengerId", "Name"], axis=1, inplace=True)

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [8]:
features = spaceship.drop(columns = ["Transported"])
features

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,B,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,F,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
2,Europa,False,A,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,A,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,F,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0
8689,Earth,True,G,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0
8690,Earth,False,G,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0
8691,Europa,False,E,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0


In [9]:
target = spaceship["Transported"]
target

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 6606, dtype: bool

In [10]:
features = pd.get_dummies(features, columns=['HomePlanet', 'Cabin', 'Destination'], drop_first=False)
features

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,...,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False,True,...,True,False,False,False,False,False,False,False,False,True
1,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True,False,...,False,False,False,False,True,False,False,False,False,True
2,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,True,...,False,False,False,False,False,False,False,False,False,True
3,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,True,...,False,False,False,False,False,False,False,False,False,True
4,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True,False,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,True,...,False,False,False,False,False,False,False,True,False,False
8689,True,18.0,False,0.0,0.0,0.0,0.0,0.0,True,False,...,False,False,False,False,False,True,False,False,True,False
8690,False,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,False,...,False,False,False,False,False,True,False,False,False,True
8691,False,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,True,...,False,False,False,True,False,False,False,True,False,False


**Perform Train Test Split**

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [12]:
normalizer = MinMaxScaler()
normalizer.fit(X_train)

X_train_norm = pd.DataFrame(normalizer.transform(X_train), columns=X_train.columns)
X_test_norm = pd.DataFrame(normalizer.transform(X_test), columns=X_test.columns)

display(X_train_norm.head())

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,...,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1.0,0.405063,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.050633,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.379747,0.0,0.00000,0.007916,0.000000,0.051276,0.000000,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.215190,0.0,0.00131,0.000000,0.046111,0.016378,0.000049,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.329114,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [13]:
bagging_class = BaggingClassifier(DecisionTreeClassifier(),
                               n_estimators=100,
                               max_samples=0.8)

In [14]:
bagging_class.fit(X_train_norm, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.8,
                  n_estimators=100)

In [15]:
pred = bagging_class.predict(X_test_norm)


In [17]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)

Accuracy: 0.7851739788199698


In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred)
print("Confusion Matrix:\n", cm)

# MATRIX EXPLANATION:
# TN | FP
# FN | TP

Confusion Matrix:
 [[524 137]
 [147 514]]


In [19]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, pred, average='weighted')  # Use 'macro' for unweighted average
print("F1 Score:", f1)


F1 Score: 0.7851616860763203


In [20]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_test, pred, average='weighted')
recall = recall_score(y_test, pred, average='weighted')
print("Precision:", precision)
print("Recall:", recall)


Precision: 0.7852392627643817
Recall: 0.7851739788199698


In [21]:
from sklearn.metrics import roc_auc_score
# For binary classification
roc_auc = roc_auc_score(y_test, bagging_class.predict_proba(X_test)[:, 1])
print("ROC AUC:", roc_auc)


ROC AUC: 0.7770260527646874


- Random Forests

In [22]:
forest = RandomForestClassifier(n_estimators=100)

In [23]:
forest.fit(X_train_norm, y_train)

RandomForestClassifier()

In [44]:
pred = forest.predict(X_test_norm)


In [45]:
accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)

cm = confusion_matrix(y_test, pred)
print("Confusion Matrix:\n", cm)

# MATRIX EXPLANATION:
# TN | FP
# FN | TP

precision = precision_score(y_test, pred, average='weighted')
recall = recall_score(y_test, pred, average='weighted')
print("Precision:", precision)
print("Recall:", recall)

f1 = f1_score(y_test, pred, average='weighted')  # Use 'macro' for unweighted average
print("F1 Score:", f1)

# For binary classification
roc_auc = roc_auc_score(y_test, bagging_class.predict_proba(X_test)[:, 1])
print("ROC AUC:", roc_auc)



0.783661119515885
Accuracy: 0.783661119515885
Confusion Matrix:
 [[528 133]
 [153 508]]
Precision: 0.7839210484718947
Recall: 0.783661119515885
F1 Score: 0.7836115937649518
ROC AUC: 0.7770260527646874


- Gradient Boosting

In [33]:
grad_class = GradientBoostingClassifier(n_estimators=100)

In [34]:
grad_class.fit(X_train_norm, y_train)

GradientBoostingClassifier()

In [35]:
pred = grad_class.predict(X_test_norm)


In [37]:
accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)

cm = confusion_matrix(y_test, pred)
print("Confusion Matrix:\n", cm)

# MATRIX EXPLANATION:
# TN | FP
# FN | TP

precision = precision_score(y_test, pred, average='weighted')
recall = recall_score(y_test, pred, average='weighted')
print("Precision:", precision)
print("Recall:", recall)

f1 = f1_score(y_test, pred, average='weighted')  # Use 'macro' for unweighted average
print("F1 Score:", f1)

# For binary classification
roc_auc = roc_auc_score(y_test, bagging_class.predict_proba(X_test)[:, 1])
print("ROC AUC:", roc_auc)



0.7859304084720121
Accuracy: 0.7859304084720121
Confusion Matrix:
 [[488 173]
 [110 551]]
Precision: 0.7885516177312959
Recall: 0.7859304084720121
F1 Score: 0.7854431486796868
Precision: 0.7885516177312959
Recall: 0.7859304084720121


- Adaptive Boosting

In [28]:
ada_class = AdaBoostClassifier(DecisionTreeClassifier(),
                            n_estimators=100)

In [29]:
ada_class.fit(X_train_norm, y_train)

c:\Users\JimiH\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=DecisionTreeClassifier(), n_estimators=100)

In [42]:
pred = ada_class.predict(X_test_norm)


In [43]:
accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)

cm = confusion_matrix(y_test, pred)
print("Confusion Matrix:\n", cm)

# MATRIX EXPLANATION:
# TN | FP
# FN | TP

precision = precision_score(y_test, pred, average='weighted')
recall = recall_score(y_test, pred, average='weighted')
print("Precision:", precision)
print("Recall:", recall)

f1 = f1_score(y_test, pred, average='weighted')  # Use 'macro' for unweighted average
print("F1 Score:", f1)

# For binary classification
roc_auc = roc_auc_score(y_test, bagging_class.predict_proba(X_test)[:, 1])
print("ROC AUC:", roc_auc)


0.7609682299546142
Accuracy: 0.7609682299546142
Confusion Matrix:
 [[465 196]
 [120 541]]
Precision: 0.7644643913300629
Recall: 0.7609682299546142
F1 Score: 0.7601756235116894
ROC AUC: 0.7770260527646874


Which model is the best and why?

Going by their Accuracy Scores, the best model appears to be gradient boosting. Gradient Boosting can achieve a high level of accuracy, especially with structured/tabular data, by reducing bias and variance over iterations. Each new tree (or weak learner) is added to correct the residuals (errors) of the previous model, leading to a refined, highly accurate model. It can also capture non-linearity in the data very well, also it is robust to outliers and handles imbalanced data well.